In [ ]:
import numpy as np
import aubio
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
DATASET_PREFIX = '../data/beatboxset1/'

class Track:
    def __init__(self, name):
        

def wav_to_array(source):
    if type(source) == str:
        with aubio.source(source) as s:
            return wav_to_array(s)
    elif type(source) == aubio.source:
        res = aubio.fvec(source.duration)
        total_read = 0
        for sample in source:
            m = sample.shape[0]
            res[total_read:total_read+m] = sample
            total_read += m
        return res, source
    
def load_audio_with_onsets(path):
    wav_array, source = wav_to_array(DATASET_PREFIX + path + '.wav')
    onsets_DR = pd.read_csv(DATASET_PREFIX + 'Annotations_DR/' + path + '.csv', names=['time', 'class'])
    onsets_HT = pd.read_csv(DATASET_PREFIX + 'Annotations_HT/' + path + '.csv', names=['time', 'class'])
    return wav_array, source, onsets_DR, onsets_HT
    
def plot_audio(wav_array, duration_seconds, onsets=None):
    fig = plt.figure(figsize=(20, 5))
    plt.plot(np.linspace(0, duration_seconds, wav_array.shape[0]), wav_array)
    if onsets is not None:
        for (idx, row) in onsets.iterrows():
            plt.axvline(x=row['time'], color='r')
    plt.ylim((-2, 2))
    plt.show()

In [ ]:
bclip, bclip_source, onsets_DR, onsets_HT = load_audio_with_onsets('snare_hex')

In [ ]:
duration_seconds = bclip_source.duration / bclip_source.samplerate
duration_seconds

In [ ]:
plot_audio(bclip, duration_seconds, onsets=onsets_DR)

In [ ]:
plot_audio(bclip, duration_seconds, onsets=onsets_HT)

In [ ]:
onset_detector = aubio.onset()

In [ ]:
bclip_new = aubio.source(DATASET_PREFIX + 'snare_hex.wav')
onsets_sec = []
onsets = []
for sample in bclip_new:
    if len(sample) == 512 and onset_detector(sample):
        onsets_sec.append(onset_detector.get_last_s())
        onsets.append(onset_detector.get_last())

classes = ['x']*len(onsets_sec)
onsets_detected = pd.DataFrame.from_dict({'time': np.array(onsets_sec), 'class': classes})

In [ ]:
bclip_new.duration, onsets_sec[-1]

In [ ]:
plot_audio(bclip, duration_seconds, onsets=onsets_detected)